In [ ]:
import  io
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [ ]:
# embedding_layer = layers.Embedding(1000,5)
# result = embedding_layer(tf.constant([1,2,3]))

# print(result.numpy())

In [ ]:
(train_data,test_data),info = tfds.load(
    "imdb_reviews",
    split = ["train","test"],
    shuffle_files = True,
    as_supervised = True,
    with_info = True
)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteWFXGO8/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteWFXGO8/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteWFXGO8/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
message = []
for text,_  in train_data:
  message.append(text.numpy())

In [ ]:
import numpy as np

In [ ]:
import nltk 
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
ps = PorterStemmer()
#with the hept of this library we will do stemming part

corpus = []
for i in range(len(message)):
  review = re.sub('[^a-zA-Z]',' ',message[i].decode('utf-8'))
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)


In [ ]:
import pickle

with open('corpus.pkl','wb') as f:
  pickle.dump(corpus,f)


In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Dense

In [ ]:
voc_size = 8000
onehot_repr =[one_hot(words,voc_size) for words in corpus]

In [ ]:
sent_length = len(max(corpus,key=len))

embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)


In [ ]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(150))
model.add(Dense(1,activation='softmax'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:
import numpy as np

In [ ]:
X_final = np.array(embedded_docs)
y_final =[]
for _,labels in train_data:
  y_final.append(labels)

In [ ]:
y_final = np.array(y_final)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.2)

In [ ]:
model.fit(X_train,y_train,epochs=5)

Epoch 1/5
625/625 [==============================] - 222s 342ms/step - loss: 0.4523 - accuracy: 0.4990
Epoch 2/5
625/625 [==============================] - 217s 348ms/step - loss: 0.2857 - accuracy: 0.4990
Epoch 3/5
625/625 [==============================] - 219s 351ms/step - loss: 0.2343 - accuracy: 0.4990
Epoch 4/5
625/625 [==============================] - 216s 345ms/step - loss: 0.1799 - accuracy: 0.4990
Epoch 5/5
625/625 [==============================] - 213s 341ms/step - loss: 0.1433 - accuracy: 0.4990


In [ ]:
model.evaluate(X_test,y_test)

157/157 [==============================] - 20s 127ms/step - loss: 0.5390 - accuracy: 0.5038


[0.5390441417694092, 0.5037999749183655]

In [ ]:
model.save_weights('./checkpoints/my_checkpoint')

NameError: ignored

In [ ]:
def create_model():
  model = Sequential()
  model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
  model.add(LSTM(150))
  model.add(Dense(1,activation='softmax'))

  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

  return model

  


In [ ]:
classifier = create_model()

In [ ]:
classifier.load_weights('./checkpoints/my_checkpoint')

In [ ]:
classifier.evaluate(X_test,y_test)

157/157 [==============================] - 21s 126ms/step - loss: 0.5390 - accuracy: 0.5038


[0.5390441417694092, 0.5037999749183655]